# IRP Integration Demo

This notebook showcases usage of the `irp_integration` module. 

### Imports & Setup

In [ ]:
%load_ext autoreload
%autoreload 2
import json
from datetime import datetime
from zoneinfo import ZoneInfo
from helpers.irp_integration import IRPClient
from helpers.sqlserver import execute_query_from_file, display_result_sets

irp_client = IRPClient()

### Configurations

In [ ]:
# Metadata / Backend Config
server_name = "databridge-1"
connection_name = "DATABRIDGE"
peril = "USEQ"
date_value = "202511"
cycle_type = "Quarterly"

# SQL Scripts Workspace EDM
workspace_edm = "WORKSPACE_EDM"

# EDM Config
edm_name = f"RM_EDM_{date_value}_{cycle_type}_{peril}" # ensure unique within IRP
treaty_name = f"{peril}_Treaty"
edm_data_version = "22"

# MRI Import Config
mapping_file_name = "mapping.json"

# Portfolio Config
portfolio_name = f"{peril}" # ensure unique within the edm
geohaz_version = "22.0"

# Analysis Config
analysis_job_name = f"{peril} Analysis"
analysis_profile_name = f"DLM {peril} v23"
output_profile_name = "Patched Portfolio Level Only (EP, ELT, Stats)"
event_rate_scheme_name = "RMS 2025 Stochastic Event Rates"
analysis_tag_name = f"{peril}_TAG"

# Grouping Config
group_name = f"RM_RDM_{date_value}_QEM_USAP"

# RDM Config
rdm_name = f"{peril}_Automated_RDM"

### Create EDM(s)

In [3]:
job_ids = irp_client.edm.submit_create_edm_jobs(
    [
        {
            "edm_name": edm_name,
            "server_name": server_name
        }
    ]
)
print(json.dumps(irp_client.job.poll_risk_data_job_batch_to_completion(job_ids), indent=2))

Polling batch risk data job ids: 23146135
Polling batch risk data job ids: 23146135
Polling batch risk data job ids: 23146135
Polling batch risk data job ids: 23146135
Polling batch risk data job ids: 23146135
Polling batch risk data job ids: 23146135
Polling batch risk data job ids: 23146135
[
  {
    "jobId": "23146135",
    "userName": "Partner - PremiumIQ key",
    "status": "FINISHED",
    "submittedAt": "2025-11-10T15:09:45.575Z",
    "startedAt": "2025-11-10T15:09:48Z",
    "endedAt": "2025-11-10T15:11:35Z",
    "name": "RM_EDM_USEQ_Quarterly_202511_Automated",
    "type": "CREATE_EDM",
    "progress": 100,
    "priority": "medium",
    "entitlement": "RI-RISKMODELER",
    "resourceGroupId": "ba672cd9-3384-4cb3-9882-bbeb5f8478c6",
    "details": {
      "resources": [],
      "summary": "CREATE_EDM is successful"
    },
    "warnings": []
  }
]


### Create Portfolio(s)

In [4]:
portfolio_ids = irp_client.portfolio.create_portfolios(
    [
        {
            "edm_name": edm_name,
            "portfolio_name": portfolio_name,
            "portfolio_number": portfolio_name,
            "description": f"{portfolio_name} created via IRP Notebook Framework"
        }
    ]
)
print(f"Created {len(portfolio_ids)} portfolios with IDs: {portfolio_ids}")

Created 1 portfolios with IDs: [1]


### MRI Import

In [8]:
import_file_prefix = f"Modeling_{date_value}_Moodys"
import_file_value = "USEQ"

if import_file_value in ["USEQ", "CBHU", "CBEQ", "USFF", "USST", "USWF", "USHU_Full", "USHU_Leak"]:
    accounts_file_name = f"{import_file_prefix}_{import_file_value}_Account.csv"
    locations_file_name = f"{import_file_prefix}_{import_file_value}_Location.csv"
elif import_file_value == "USFL_Other":
    accounts_file_name = f"{import_file_prefix}_{cycle_type}_OtherFld_Account.csv"
    locations_file_name = f"{import_file_prefix}_{cycle_type}_OtherFld_Location.csv"
elif import_file_value == "USFL_Excess":
    accounts_file_name = f"{import_file_prefix}_{cycle_type}_ExcessFld_Account.csv"
    locations_file_name = f"{import_file_prefix}_{cycle_type}_ExcessFld_Location.csv"
elif import_file_value == "USFL_Commercial":
    accounts_file_name = f"{import_file_prefix}_{cycle_type}_ComFld_Account.csv"
    locations_file_name = f"{import_file_prefix}_{cycle_type}_ComFld_Location.csv"

import_response = irp_client.mri_import.import_from_files(
    edm_name=edm_name,
    portfolio_name=portfolio_name,
    accounts_file=accounts_file_name,
    locations_file=locations_file_name,
    mapping_file="mapping.json"
)

print(json.dumps(import_response, indent=2))

Creating AWS bucket...
AWS bucket created!
Access credentials received
Uploading file /home/jovyan/workspace/workflows/_Tools/files/working_files/Modeling_202511_Moodys_USEQ_Account.csv to s3...
File uploaded!
Uploading file /home/jovyan/workspace/workflows/_Tools/files/working_files/Modeling_202511_Moodys_USEQ_Location.csv to s3...
File uploaded!
Submitting workflow request...
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/23146148
Workflow status: PENDING; Percent complete: 0
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/23146148
Workflow status: RUNNING; Percent complete: 0
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/23146148
Workflow status: RUNNING; Percent complete: 0
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/23146148
Workflow status: RUNNING; Percent complete: 0
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/23146148
Workflow status

### Create Treaty

In [9]:
treaty_ids = irp_client.treaty.create_treaties(
    [
        {
            "edm_name": edm_name,
            "treaty_name": treaty_name,
            "treaty_number": treaty_name,
            "treaty_type": "Working Excess",
            "per_risk_limit": 30000000.00,
            "occurrence_limit": 90000000.00,
            "attachment_point": 2000000.00,
            "inception_date": "2025-01-01T00:00:00.000Z",
            "expiration_date": "2030-01-01T00:00:00.000Z",
            "currency_name": "US Dollar",
            "attachment_basis": "Losses Occurring",
            "attachment_level": "Location",
            "pct_covered": 100.0,
            "pct_placed": 95.00,
            "pct_share": 100.00,
            "pct_retention": 100.00,
            "premium": 0.00,
            "num_reinstatements": 99,
            "pct_reinstatement_charge": 0.00,
            "aggregate_limit": 0.00,
            "aggregate_deductible": 0.00,
            "priority": 1
        }
    ]
)
print(f"Created {len(treaty_ids)} with IDs: {treaty_ids}")

Created 1 with IDs: [1]


### Upgrade EDM Version

In [10]:
upgrade_job_ids = irp_client.edm.submit_upgrade_edm_data_version_jobs(
    [
        {
            "edm_name": edm_name,
            "edm_version": edm_data_version
        }
    ]
)
print(json.dumps(irp_client.edm.poll_data_version_upgrade_job_batch_to_completion(upgrade_job_ids), indent=2))

Polling batch upgrade edm version job ids: 23146158
Polling batch upgrade edm version job ids: 23146158
Polling batch upgrade edm version job ids: 23146158
[
  {
    "id": 23146158,
    "userName": "Partner - PremiumIQ key",
    "status": "FINISHED",
    "submitTime": "2025-11-10T15:30:17.273Z",
    "startTime": "2025-11-10T15:30:18Z",
    "endTime": "2025-11-10T15:30:37Z",
    "name": "RM_EDM_USEQ_Quarterly_202511_Automated",
    "type": "EDM_DATA_UPGRADE",
    "progress": 100,
    "priority": "medium",
    "entitlement": "RI-RISKMODELER",
    "resourceGroupId": "ba672cd9-3384-4cb3-9882-bbeb5f8478c6",
    "jobs": [
      {
        "id": "a5feb46b-8e85-4f8a-b4ff-aba145882127",
        "taskId": 1,
        "workflowId": 23146158,
        "status": "Succeeded",
        "submitTime": "2025-11-10T15:30:17.919Z",
        "createdAt": "2025-11-10T15:30:17.260Z",
        "name": "EDM_DATA_UPGRADE",
        "input": {
          "name": "EDM_DATA_UPGRADE"
        },
        "output": {
        

### GeoHaz

In [11]:
geohaz_job_ids = irp_client.portfolio.submit_geohaz_jobs(
    [
        {
            "edm_name": edm_name,
            "portfolio_name": portfolio_name,
            "version": geohaz_version,
            "hazard_eq": True,
            "hazard_ws": True
        }
    ]
)
print(json.dumps(irp_client.portfolio.poll_geohaz_job_batch_to_completion(geohaz_job_ids), indent=2))

Polling batch geohaz job ids: 23146165
Polling batch geohaz job ids: 23146165
Polling batch geohaz job ids: 23146165
Polling batch geohaz job ids: 23146165
Polling batch geohaz job ids: 23146165
Polling batch geohaz job ids: 23146165
Polling batch geohaz job ids: 23146165
Polling batch geohaz job ids: 23146165
Polling batch geohaz job ids: 23146165
Polling batch geohaz job ids: 23146165
Polling batch geohaz job ids: 23146165
Polling batch geohaz job ids: 23146165
Polling batch geohaz job ids: 23146165
Polling batch geohaz job ids: 23146165
Polling batch geohaz job ids: 23146165
Polling batch geohaz job ids: 23146165
Polling batch geohaz job ids: 23146165
Polling batch geohaz job ids: 23146165
Polling batch geohaz job ids: 23146165
Polling batch geohaz job ids: 23146165
Polling batch geohaz job ids: 23146165
Polling batch geohaz job ids: 23146165
Polling batch geohaz job ids: 23146165
Polling batch geohaz job ids: 23146165
Polling batch geohaz job ids: 23146165
Polling batch geohaz job 

### Portfolio Mapping

In [17]:
edms = irp_client.edm.search_edms(filter=f"exposureName=\"{edm_name}\"")
if (len(edms) != 1):
    raise Exception(f"Found {len(edms)} EDMs with name {edm_name}; expected 1")
edm = edms[0]
exposure_id = edm['exposureId']
edm_full_name = edm['databaseName']
print(f'EDM Full Name: {edm_full_name}')

portfolios = irp_client.portfolio.search_portfolios(exposure_id, f"portfolioName=\"{portfolio_name}\"")
if (len(portfolios) != 1):
    raise Exception(f"Found {len(edms)} Portfolios with name {portfolio_name}; expected 1")
portfolio_id = portfolios[0]['portfolioId']
print(f'Portfolio ID: {portfolio_id}')
print()

sql_script_name = f"2b_Query_To_Create_Sub_Portfolios_{peril}_RMS_BackEnd.sql"
params = {
    "EDM_FULL_NAME": edm_full_name,
    "PORTFOLIO_ID": portfolio_id,
    "DATETIME_VALUE": datetime.now(ZoneInfo("America/New_York")).strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
}

result = execute_query_from_file(
    file_path = f'portfolio_mapping/{sql_script_name}',
    params = params,
    connection = connection_name
)

display_result_sets(result)

EDM Full Name: RM_EDM_USEQ_Quarterly_202511_Automated_Wiyi
Portfolio ID: 1

Reading SQL script ...
Parameterizing ...
Executing SQL query: 2b_Query_To_Create_Sub_Portfolios_USEQ_RMS_BackEnd.sql
✓ Retrieved 1 result set(s)

QUERY RESULTS: 1 result set(s)


--------------------------------------------------------------------------------
Result Set 1 of 1
--------------------------------------------------------------------------------
Rows: 7 | Columns: 7



,PORTINFOID,PORTNUM,PORTNAME,CREATEDATE,DESCRIPT,AccountGroupCount,PortfolioAccountCount
0,5,USEQ_CHFS,USEQ_CHFS,2025-11-10 10:52:48.183,USEQ_CHFS,344,344
1,7,USEQ_Clay_21st,USEQ_Clay_21st,2025-11-10 10:52:48.183,USEQ_Clay_21st,148768,148768
2,8,USEQ_Clay_Homes,USEQ_Clay_Homes,2025-11-10 10:52:48.183,USEQ_Clay_Homes,136315,136315
3,4,USEQ_Condo,USEQ_Condo,2025-11-10 10:52:48.183,USEQ_Condo,144,144
4,2,USEQ_Manufactured,USEQ_Manufactured,2025-11-10 10:52:48.183,USEQ_Manufactured,196767,196767
5,6,USEQ_Other,USEQ_Other,2025-11-10 10:52:48.183,USEQ_Other,0,0
6,3,USEQ_Renters,USEQ_Renters,2025-11-10 10:52:48.183,USEQ_Renters,34439,34439


### Control Totals

In [13]:
workspace_edms = irp_client.edm.search_edms(filter=f"exposureName=\"{workspace_edm}\"")
if (len(edms) != 1):
    raise Exception(f"Found {len(edms)} EDMs with name {workspace_edm}; expected 1")
workspace_edm = workspace_edms[0]
workspace_edm_full_name = workspace_edm['databaseName']
print(f'Workspace EDM Full Name: {workspace_edm_full_name}')
print()

Workspace EDM Full Name: WORKSPACE_EDM_uXJm



In [18]:
# EDM Control Totals

sql_script_name = '3d_RMS_EDM_CONTROL_Totals.sql'
params = {
    "WORKSPACE_EDM": workspace_edm_full_name,
    "CYCLE_TYPE": cycle_type,
    "DATE_VALUE": date_value
}

result = execute_query_from_file(
    file_path = f'control_totals/{sql_script_name}',
    params = params,
    connection = connection_name
)

display_result_sets(result)

Reading SQL script ...
Parameterizing ...
Executing SQL query: 3d_RMS_EDM_CONTROL_Totals.sql
✓ Retrieved 10 result set(s)

QUERY RESULTS: 10 result set(s)


--------------------------------------------------------------------------------
Result Set 1 of 10
--------------------------------------------------------------------------------
Rows: 7 | Columns: 6



,PORTNAME,PolicyCount,PolicyLimit,PolicyPremium,AttachmentPoint,PolicyDeductible
0,USEQ_Automated,516777,61568158484.00,565524244.00,0.00,0.00
1,USEQ_CHFS,344,362956599.00,1359514.00,0.00,0.00
2,USEQ_Clay_21st,148768,17232894141.00,158768016.00,0.00,0.00
3,USEQ_Clay_Homes,136315,19141446773.00,170120715.00,0.00,0.00
4,USEQ_Condo,144,21756768.00,149376.00,0.00,0.00
5,USEQ_Manufactured,196767,23973540757.00,227029441.00,0.00,0.00
6,USEQ_Renters,34439,835563446.00,8097182.00,0.00,0.00



--------------------------------------------------------------------------------
Result Set 2 of 10
--------------------------------------------------------------------------------
Rows: 7 | Columns: 3



,PORTNAME,LocationCountDistinct,LocationCountCampus
0,USEQ_Automated,516777,516777
1,USEQ_CHFS,344,344
2,USEQ_Clay_21st,148768,148768
3,USEQ_Clay_Homes,136315,136315
4,USEQ_Condo,144,144
5,USEQ_Manufactured,196767,196767
6,USEQ_Renters,34439,34439



--------------------------------------------------------------------------------
Result Set 3 of 10
--------------------------------------------------------------------------------
Rows: 7 | Columns: 3



,PORTNAME,TotalReplacementValue,LocationLimit
0,USEQ_Automated,61568158484.00,61359987448.00
1,USEQ_CHFS,362956599.00,166085409.00
2,USEQ_Clay_21st,17232894141.00,17232894141.00
3,USEQ_Clay_Homes,19141446773.00,19141446773.00
4,USEQ_Condo,21756768.00,19410598.00
5,USEQ_Manufactured,23973540757.00,23973540757.00
6,USEQ_Renters,835563446.00,826609770.00



--------------------------------------------------------------------------------
Result Set 4 of 10
--------------------------------------------------------------------------------
Rows: 7 | Columns: 2



,PORTNAME,LocationDeductible
0,USEQ_Automated,4200801551.00
1,USEQ_CHFS,24789271.00
2,USEQ_Clay_21st,1226467153.00
3,USEQ_Clay_Homes,1044894547.00
4,USEQ_Condo,2009495.00
5,USEQ_Manufactured,1798986692.00
6,USEQ_Renters,103654393.00



--------------------------------------------------------------------------------
Result Set 5 of 10
--------------------------------------------------------------------------------
Empty DataFrame
Columns: ['PORTNAME', 'PolicyCount', 'PolicyPremium', 'PolicyLimit_NonCommercialFlood', 'AttachmentPoint', 'PolicyDeductible']

--------------------------------------------------------------------------------
Result Set 6 of 10
--------------------------------------------------------------------------------
Empty DataFrame
Columns: ['USFL_Commercial_PolicyLimit']

--------------------------------------------------------------------------------
Result Set 7 of 10
--------------------------------------------------------------------------------
Empty DataFrame
Columns: ['PORTNAME', 'Policy_Sublimit']

--------------------------------------------------------------------------------
Result Set 8 of 10
--------------------------------------------------------------------------------
Empty DataFrame

In [23]:
# GeoHaz Control Totals

sql_script_name = '3e_GeocodingSummary.sql'
params = {
    "WORKSPACE_EDM": workspace_edm_full_name,
    "CYCLE_TYPE": cycle_type,
    "DATE_VALUE": date_value
}

result = execute_query_from_file(
    file_path = f'control_totals/{sql_script_name}',
    params = params,
    connection = connection_name
)

display_result_sets(result)

Reading SQL script ...
Parameterizing ...
Executing SQL query: 3e_GeocodingSummary.sql
✓ Retrieved 1 result set(s)

QUERY RESULTS: 1 result set(s)


--------------------------------------------------------------------------------
Result Set 1 of 1
--------------------------------------------------------------------------------
Rows: 6 | Columns: 7



,DBname,PORTNAME,GeoResolutionCode,GeocodeDescription,RiskCount,TIV,TRV
0,RM_EDM_USEQ_Quarterly_202511_Automated_Wiyi,USEQ_Automated,0,Null,1,121940.00,121940.00
1,RM_EDM_USEQ_Quarterly_202511_Automated_Wiyi,USEQ_Automated,1,Coordinate,482615,57104105908.00,57307871863.00
2,RM_EDM_USEQ_Quarterly_202511_Automated_Wiyi,USEQ_Automated,2,Street address,13617,1720289855.00,1724018076.00
3,RM_EDM_USEQ_Quarterly_202511_Automated_Wiyi,USEQ_Automated,4,Street Name,4198,557073685.00,557109685.00
4,RM_EDM_USEQ_Quarterly_202511_Automated_Wiyi,USEQ_Automated,5,Postcode,16328,1976786566.00,1977427426.00
5,RM_EDM_USEQ_Quarterly_202511_Automated_Wiyi,USEQ_Automated,7,City/Town,15,1609494.00,1609494.00


### Execute Multiple Analyses

In [ ]:
analysis_job_ids = irp_client.analysis.submit_portfolio_analysis_jobs(
    [
        {
            "edm_name": edm_name,
            "job_name": "CBHU 2023 Rates",
            "portfolio_name": portfolio_name,
            "analysis_profile_name": analysis_profile_name,
            "output_profile_name": output_profile_name,
            "event_rate_scheme_name": "RMS 2023 Stochastic Event Rates",
            "treaty_names": [treaty_name],
            "tag_names": [analysis_tag_name]
        },
        {
            "edm_name": edm_name,
            "job_name": "CBHU 2025 Rates",
            "portfolio_name": portfolio_name,
            "analysis_profile_name": analysis_profile_name,
            "output_profile_name": output_profile_name,
            "event_rate_scheme_name": "RMS 2025 Stochastic Event Rates",
            "treaty_names": [treaty_name],
            "tag_names": [analysis_tag_name]
        }
    ]
)
analysis_batch_response = irp_client.analysis.poll_analysis_job_batch_to_completion(analysis_job_ids)
print(json.dumps(analysis_batch_response, indent=2))

### Group Analyses

In [ ]:
analysis_names = ["CBHU 2023 Rates", "CBHU 2025 Rates"]
grouping_job_ids = irp_client.analysis.submit_analysis_grouping_jobs(
    [
        {
            "group_name": group_name,
            "analysis_names": analysis_names
        }
    ]
)
grouping_response = irp_client.analysis.poll_analysis_grouping_job_batch_to_completion(grouping_job_ids)
print(json.dumps(grouping_response, indent=2))

### Export Analyses to RDM

In [ ]:
analysis_names = ["CBHU Group", "CBHU 2023 Rates", "CBHU 2025 Rates"]
rdm_export_response = irp_client.rdm.export_analyses_to_rdm(
    server_name=server_name,
    rdm_name=rdm_name,
    analysis_names=analysis_names
)
print(json.dumps(rdm_export_response, indent=2))

### Delete EDM

In [ ]:
print(json.dumps(irp_client.edm.delete_edm(edm_name), indent=2))